# Preconditioning


## Nordic Graduate Coarse on Computational Mathematical Modeling
### Miroslav Kuchta (miroslav.kuchta@gmail.com)

The following notebook contains set of exercises accompanying the lecture. Prerequisites for executing the code are working installations of [FEniCS](https://fenicsproject.org/download/) and [cbc.block](https://bitbucket.org/fenics-apps/cbc.block). The lecture material and some of the exercises were inspired by the following books and papers (I tried to list them in the order of relavance)

1. [Preconditioning discretizations of systems of partial differential equations](http://onlinelibrary.wiley.com/doi/10.1002/nla.716/abstract)
2. [Preconditioning and the Conjugate Gradient Method in the Context of Solving PDEs](http://epubs.siam.org/doi/book/10.1137/1.9781611973846)
3. [From Functional Analysis to Iterative Methods](http://epubs.siam.org/doi/abs/10.1137/070706914)
4. [A Note on Preconditioners and Scalar Products in Krylov Subspace Methods for Self-adjoint Problems in Hilbert Space ](etna.mcs.kent.edu/vol.41.2014/pp13-20.dir/pp13-20.pdf)
5. [The Mathematical Theory of Finite Element Methods](https://books.google.no/books/about/The_Mathematical_Theory_of_Finite_Elemen.html?id=ci4c_R0WKYYC&redir_esc=y)
6. [An Introduction to the Conjugate Gradient Method Without the Agonizing Pain](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf)
7. [A Multigrid Tutorial](http://epubs.siam.org/doi/book/10.1137/1.9780898719505)
8. [Realistic Eigenvalue Bounds for the Galerkin Mass Matrix](https://academic.oup.com/imajna/article-abstract/7/4/449/841299/Realistic-Eigenvalue-Bounds-for-the-Galerkin-Mass?redirectedFrom=PDF)

In addition [Golub's](https://jhupbooks.press.jhu.edu/content/matrix-computations) book covers many more classical iterative and Krylov subspace methods that were not covered in the lecture. [Strang](http://epubs.siam.org/doi/abs/10.1137/1028182)'s book has a very readable introduction to multigrid while [Quarteroni](http://www.springer.com/gp/book/9788847055216)'s book is a nice read on domain decomposition techniques.

# Scaling of the stiffness matrix

In [ ]:
from dolfin import *

def HyperCubeMesh(dim, ncells):
    return {1: UnitIntervalMesh,
            2: UnitSquareMesh,
            3: UnitCubeMesh}[dim](*(ncells, )*dim)

Let's verify our prediction that the condition number of the stiffness matrix is $\mathcal{O}(h^{-2})$

In [ ]:
import numpy as np

def stiffness_cond(dim, n):
    '''
    Condition number of matrix corresponding to
    
        -\Delta u = f in [0, 1]^dim
                u = 0 on boundary
    
    discretized with P1 elements.
    '''
    mesh = HyperCubeMesh(dim, n)
    # Want all eigenvalues so keep it small
    assert mesh.num_vertices() < 5000 
    
    mesh = HyperCubeMesh(dim, n)
    h = mesh.hmin() 
    
    V = FunctionSpace(mesh, 'CG', 1)
    u = TrialFunction(V)
    v = TestFunction(V)
    
    a = inner(grad(u), grad(v))*dx
    L = inner(Constant(0), v)*dx
    bc = DirichletBC(V, Constant(0), 'on_boundary')
    
    A, _ = assemble_system(a, L, bc)
    A = A.array()
    
    lmin, lmax = np.sort(np.linalg.eigvalsh(A))[[0, -1]]
    
    return h, lmax/lmin

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (14, 8)
matplotlib.rcParams['font.size'] = 14

In [ ]:
import matplotlib.pyplot as plt

dim = 1
h, cond = np.array([stiffness_cond(1, 2**i) for i in range(2, 11)]).T

plt.loglog(h, cond, label='$\kappa$')
plt.loglog(h, h**(-2), label='$h^{-2}$')
plt.legend(loc='best')

## Iterative solvers for Poisson problem

In [ ]:
def poisson(dim, n, sigma=Constant(1), epsilon=Constant(1)):
    '''
    Linear system corresponding to P1 discretization of
    
        -\Delta u + sigma*u = f in [0, 1]^dim
                          u = 0 on D = {x = boundary, x[0] == 0}
          du/dn + epsilon*u = 0 on boundary\D
    '''
    mesh = HyperCubeMesh(dim, n)
    
    V = FunctionSpace(mesh, 'CG', 1)
    u = TrialFunction(V)
    v = TestFunction(V)
    
    bc = DirichletBC(V, Constant(0), 'near(x[0], 0)')
    boundaries = FacetFunction('size_t', mesh, 1)
    # So that Neumann is the rest, 1
    CompiledSubDomain('near(x[0], 0)').mark(boundaries, 0)
    # Measure for Neumann
    dsN = Measure('ds', domain=mesh, subdomain_data=boundaries, subdomain_id=1)
    
    a = inner(grad(u), grad(v))*dx + inner(sigma*u, v)*dx + inner(epsilon*u, v)*dsN
    L = inner(Constant(0), v)*dx
    
    A, b = assemble_system(a, L, bc)
    
    return A, b, V

We showed the the above problem is well-posed with $V=H^1_{0, \Gamma_D}$ considered with both the $H^1$ norm
or the $H^1$ seminorm. Let's define the corresponding Riesz maps. Note that we will use algebraic multigrid to define
their approximate inverse. 

**Exercise**
- Try other ways to approximate the mapping. ML, ILU, [etc](https://bitbucket.org/fenics-apps/cbc.block/src/master/block/algebraic/petsc/precond.py?at=master&fileviewer=file-view-default). Look at iteration count as well as the cost.

In [ ]:
from block.algebraic.petsc import AMG

def riesz_H1(V):
    '''Operator corresponding to approx. Riesz map w.r.t to H1 norm. '''
    u = TrialFunction(V)
    v = TestFunction(V)
    
    bc = DirichletBC(V, Constant(0), 'near(x[0], 0)')
    a = inner(grad(u), grad(v))*dx + inner(u, v)*dx
    L = inner(Constant(0), v)*dx
    
    A, _ = assemble_system(a, L, bc)
    return AMG(A)

In [ ]:
def riesz_H10(V):
    '''Operator corresponding to approx. Riesz map w.r.t to H1 seminorm. '''
    u = TrialFunction(V)
    v = TestFunction(V)
    
    bc = DirichletBC(V, Constant(0), 'near(x[0], 0)')
    a = inner(grad(u), grad(v))*dx
    L = inner(Constant(0), v)*dx
    
    A, _ = assemble_system(a, L, bc)
    return AMG(A)

We use implementation of Richardson algorithm from cbc.block

In [ ]:
from block.iterative import Richardson

riesz = riesz_H10

dim = 2
for n in (8, 16, 32, 64, 128, 256, 512):
    A, b, V = poisson(dim, n)
    B = riesz(V)
    
    uh = Function(V)
    x = uh.vector()
    # Start from random initial guess
    x.set_local(np.random.rand(x.local_size())) 
    
    # Step size \rho
    rho = 0.5
    Ainv = Richardson(A=A, precond=rho*B, initial_guess=x, tolerance=1e-4)
    x[:] = Ainv*b
    niters = len(Ainv.residuals)-1
    
    print 'n = %d, dofs = %d, niters = %d' % (n, A.size(0), niters)

**Exericise**
- How does parameter of algorithm $\rho$ effect convergence behavior?
- Try setting `B=1`, i.e. no preconditioner. What happens?
- What happens when you change the values of problem parameters $\sigma$ and $\epsilon$

What about Jacobi? This method is studied in great detail in reference [7] together with its weighted version. In short, stiffness matrices do not have the right spectral properties for it to work.

In [ ]:
from petsc4py import PETSc

def Diag(A):
    '''Diagonal matrix diag(A)'''
    dvec = as_backend_type(A).mat().getDiagonal()

    diagAinv = PETSc.Mat().createAIJ(dvec.size, nnz=1)
    diagAinv.setDiagonal(dvec)
    return PETScMatrix(diagAinv)


def InvDiag(A):
    '''Diagonal matrix diag(A)^-1'''
    dvec = as_backend_type(A).mat().getDiagonal()
    dvec.reciprocal()

    diagAinv = PETSc.Mat().createAIJ(dvec.size, nnz=1)
    diagAinv.setDiagonal(dvec)
    return PETScMatrix(diagAinv)

In [ ]:
dim = 1
for n in (8, 16, 32, 64, 128, 256, 512):
    A, b, V = poisson(dim, n)
    B = InvDiag(A)
    
    uh = Function(V)
    x = uh.vector()
    # Start from random initial guess
    x.set_local(np.random.rand(x.local_size())) 
    
    Ainv = Richardson(A=A, precond=2./3*B, initial_guess=x, tolerance=1e-4)
    x[:] = Ainv*b
    niters = len(Ainv.residuals)-1
    
    print 'n = %d, dofs = %d, niters = %d' % (n, A.size(0), niters)

**Exercise**
- Using what you heard on the lecture and and reference [8] do you expect Jacobi to work for the problem of finding the $L^2$ projection?

In [ ]:
def projection(dim, n):
    '''
    Linear system corresponding to L^2 projection
    '''
    mesh = HyperCubeMesh(dim, n)
    
    V = FunctionSpace(mesh, 'CG', 1)
    u = TrialFunction(V)
    v = TestFunction(V)
    
    a = inner(u, v)*dx
    L = inner(Constant(1), v)*dx
    
    A, b = assemble_system(a, L)
    
    return A, b, V

In [ ]:
from scipy.linalg import eigvalsh

dim = 3
for n in (2, 4, 8, 16, 32):
    M, b, V = projection(dim, n)
    B = InvDiag(M)
    
    # Look at the condition number of the system
    if dim < 3 or V.dim() < 5000:
        Binv = Diag(M)
        lmin, lmax = np.sort(eigvalsh(M.array(), Binv.array()))[[0, -1]]
        print 'lmin = %g, lmax = %g, cond = %g' % (lmin, lmax, lmax/lmin)
    
    uh = Function(V)
    x = uh.vector()
    # Start from smooth initial guess, 0
    # Note the scaling 2/3.
    Ainv = Richardson(A=M, precond=2./3*B, initial_guess=x, tolerance=1e-4, maxiter=10000)
    x[:] = Ainv*b
    niters = len(Ainv.residuals)-1
    
    print (dim<3 or V.dim() < 5000)*'\t', 'n = %d, dofs = %d, niters = %d' % (n, M.size(0), niters)

Now, let's see about some parameter free methods. We begin with steepest descent.

In [ ]:
def steepest_descent(A, x, b, B, tolerance, maxiter=200):
    '''Preconditioned steepest descent method.'''
    r = b - A*x
    error = sqrt(r.inner(r))

    iter = 0
    while error > tolerance and iter < maxiter:
        
        p = B*r
        Ap = A*p 
        rho = r.inner(p)/Ap.inner(p)
        x += rho*p
        r = b - A*x

        error = sqrt(r.inner(r))
        iter += 1

    return iter

In [ ]:
riesz = riesz_H10

dim = 2
for n in (8, 16, 32, 64, 128, 256, 512):
    A, b, V = poisson(dim, n)
    B = riesz(V)
    
    uh = Function(V)
    x = uh.vector()
    # Start from random initial guess
    x.set_local(np.random.rand(x.local_size())) 
    
    niters = steepest_descent(A, x, b, B, tolerance=1e-4, maxiter=200)
    
    print 'n = %d, dofs = %d, niters = %d' % (n, A.size(0), niters)

Finally we shall see about the superiority of conjugate gradient.

In [ ]:
from block.iterative import ConjGrad

riesz = riesz_H10

dim = 2
for n in (8, 16, 32, 64, 128, 256, 512):
    A, b, V = poisson(dim, n)
    B = riesz(V)
    
    uh = Function(V)
    x = uh.vector()
    # Start from random initial guess
    x.set_local(np.random.rand(x.local_size())) 
    
    Ainv = ConjGrad(A=A, precond=B, initial_guess=x, tolerance=1e-4)
    x[:] = Ainv*b
    niters = len(Ainv.residuals)-1
    
    print 'n = %d, dofs = %d, niters = %d' % (n, A.size(0), niters)

**Exercise**
- How do the CG and steepest descent algorithms perform with some simpler preconditioner or even without any preconditioner?
- Changing the preconditioner for something else, e.g. Jacobi, Gauss-Seidel, see [here](https://bitbucket.org/fenics-apps/cbc.block/src/8447b459156459b5a6f29ce8129fc07c2dc644cd/block/algebraic/petsc/precond.py?at=master&fileviewer=file-view-default) in all the tests below is a useful experience

## Iterative solvers for convection-diffusion

In most text books exposition of iterative methods typically revolves around symmetric, positive-definite matrices. We break away from this tradition and introduce asymmetry into our Poisson problem.

In [ ]:
def cd_system_Galerkin(n, epsilon=1):
    '''
    Linear system corresponding to P1 discretization of convection-diffusion problem
    
        -\epsilon \Delta u + mag\beta \cdot \nabla u = f
                                          u = 0 on inflow
                                      du/dn = 0 on outflow
    '''
    mesh = UnitSquareMesh(n, n)
    V = FunctionSpace(mesh, 'CG', 1)
    u = TrialFunction(V)
    v = TestFunction(V)
    
    bc = DirichletBC(V, Constant(0), 'near(x[0], 0)')
    
    # Divergence free wind velocity
    beta = Expression(('A*x[1]*(1-x[1])', '0'), degree=2, A=1.)
    
    f = Constant(1)
    
    a = Constant(epsilon)*inner(grad(u), grad(v))*dx + inner(dot(beta, grad(u)), v)*dx
    L = inner(f, v)*dx
    
    A = assemble(a)
    b = assemble(L)
    bc.apply(A, b)
    
    return A, b, V

**Exercise**
- The following works with given parameter $\epsilon$. See how sensitive the method parameter $\rho$ is with respect to $\epsilon$.

In [ ]:
riesz = riesz_H1

dim = 2
for n in (8, 16, 32, 64, 128):
    A, b, V = cd_system_Galerkin(n, epsilon=1E0)
    B = riesz(V)
    
    uh = Function(V)
    x = uh.vector()
    # Start from random initial guess
    x.set_local(np.random.rand(x.local_size())) 
    
    # Step size \rho
    rho = 0.8
    Ainv = Richardson(A=A, precond=rho*B, initial_guess=x, tolerance=1e-4)
    x[:] = Ainv*b
    niters = len(Ainv.residuals)-1
    
    print 'n = %d, dofs = %d, niters = %d' % (n, A.size(0), niters)

With that experince let's use some Krylov method, where no tuning is required. We can't used conjugate gradients for convection diffusion problem since the problem is not symmetric. The lack of symmetry also excludes minimal residual method (MINRES). We are left with GMRES. The code below should work well with given $\epsilon$. 

In [ ]:
from block.iterative import LGMRES

riesz = riesz_H1

dim = 2
for n in (8, 16, 32, 64, 128, 256, 512):
    A, b, V = cd_system_Galerkin(n, epsilon=1E-3)
    B = riesz(V)
    
    uh = Function(V)
    x = uh.vector()
    # Start from random initial guess
    x.set_local(np.random.rand(x.local_size())) 
    
    # Step size \rho
    Ainv = LGMRES(A=A, precond=B, initial_guess=x, tolerance=1e-6)
    x[:] = Ainv*b
    niters = len(Ainv.residuals)-1
    
    print 'n = %d, dofs = %d, niters = %d' % (n, A.size(0), niters)

**Exercise**
- What happens when you increase the parameter's value? Note that GMRES is memory intense so consider smaller matrices.
- To simplify things (forget abour convegence of iterative solver), try switching to direct solver and see how the approximation properties are for different $\epsilon$. To do this you should implement a simpler ($1d$ is fine) problem for which you will have at your disposal an analytical solution.

One way to deal with dominating convection is to introduce diffusion. One possibility is the [Streamline-Upwind-Petrov-Galerkin method](http://www.sciencedirect.com/science/article/pii/0045782582900718)

In [ ]:
def cd_system_SUPG(n, epsilon=1.):
    '''
    Linear system corresponding to P1-SUPG discretization of convection-diffusion problem
    
        -\epsilon \Delta u + mag\beta \cdot \nabla u = f
                                          u = 0 on inflow
                                      du/dn = 0 on outflow
                                      
    and the related Riesz map preconditioner
    '''
    mesh = UnitSquareMesh(n, n)
    V = FunctionSpace(mesh, 'CG', 1)
    u = TrialFunction(V)
    v = TestFunction(V)
    
    bc = DirichletBC(V, Constant(0), 'near(x[0], 0)')
    
    # Divergence free wind velocity
    beta = Expression(('x[1]*(1-x[1])', '0'), degree=2)
    
    f = Constant(1)
    
    a = Constant(epsilon)*inner(grad(u), grad(v))*dx + inner(dot(beta, grad(u)), v)*dx
    L = inner(f, v)*dx
    
    h = mesh.hmin()
    beta_max = abs(0.25)
    delta = Constant(h/2./beta_max)
    # SUPG Stabilization
    a += delta*inner(dot(beta, grad(u)), dot(beta, grad(v)))*dx
    L += inner(f, delta*dot(beta, grad(v)))*dx
    
    A = assemble(a)
    b = assemble(L)
    bc.apply(A, b)
    
    # -----
    
    a = Constant(epsilon)*inner(grad(u), grad(v))*dx +\
        delta*inner(dot(beta, grad(u)), dot(beta, grad(v)))*dx
    L = inner(Constant(0), v)*dx

    B, _ = assemble_system(a, L, bc)
    assert A.size(0) == A.size(1) == B.size(0) == B.size(1) == b.size()
    B = AMG(B)
    
    return A, b, B, V

Here we verify that $h$ robustness. 

In [ ]:
for n in (8, 16, 32, 64, 128, 256, 512):
    A, b, B, V = cd_system_SUPG(n, epsilon=1E-4)
    
    uh = Function(V)
    x = uh.vector()
    # Start from random initial guess
    x.set_local(np.random.rand(x.local_size())) 
    
    # Step size \rho
    Ainv = LGMRES(A=A, precond=B, initial_guess=x, tolerance=1e-4)
    x[:] = Ainv*b
    niters = len(Ainv.residuals)-1
    
    print 'n = %d, dofs = %d, niters = %d' % (n, A.size(0), niters)

**Exercise**
- How are the iterations effected by changes in $\epsilon$. You might want to consult [3]
- And what about approximation properties?**.

_Please report to me any 'bugs' you find in the above code. They are left there for purpose (secret features) and I want to make sure they are all uncovered ;)_